## Preprocessing Depth Data
### NeWCRFs expects metric depth (meters), but RealSense depth might be in millimeters. Convert it:

In [ ]:
# import cv2
# import numpy as np
# import os

# DATASET_PATH = "./datasets/custom_data/rgb_depth"
# CONVERTED_DEPTH_PATH = "./datasets/custom_data/custom_data_converted/"

# # Ensure output folder exists
# os.makedirs(CONVERTED_DEPTH_PATH, exist_ok=True)

# # Convert depth images to meters
# for folder in os.listdir(DATASET_PATH):
#     depth_folder = os.path.join(DATASET_PATH, folder, "depth")
#     output_depth_folder = os.path.join(CONVERTED_DEPTH_PATH, folder, "depth")
#     os.makedirs(output_depth_folder, exist_ok=True)

#     for file in os.listdir(depth_folder):
#         depth_map = cv2.imread(os.path.join(depth_folder, file), cv2.IMREAD_UNCHANGED).astype(np.float32)
#         depth_map /= 1000.0  # Convert mm to meters
#         cv2.imwrite(os.path.join(output_depth_folder, file), (depth_map * 1000).astype(np.uint16))


In [16]:
import cv2
import numpy as np
import os

DATASET_PATH = "./datasets/custom_data/rgb_depth"

# Convert depth images to meters and store them inside the same folder structure
for folder in os.listdir(DATASET_PATH):
    folder_path = os.path.join(DATASET_PATH, folder)  # Get the subfolder path
    depth_folder = os.path.join(folder_path, "depth")  # Original depth folder
    converted_depth_folder = os.path.join(folder_path, "depth_converted")  # New converted depth folder

    # Ensure converted depth folder exists inside the same subfolder
    os.makedirs(converted_depth_folder, exist_ok=True)

    for file in os.listdir(depth_folder):
        depth_map = cv2.imread(os.path.join(depth_folder, file), cv2.IMREAD_UNCHANGED).astype(np.float32)
        depth_map /= 1000.0  # Convert mm to meters
        cv2.imwrite(os.path.join(converted_depth_folder, file), (depth_map * 1000).astype(np.uint16))

print("Depth conversion completed! Converted files are stored inside each original folder.")


Depth conversion completed! Converted files are stored inside each original folder.


## Adjust Dataset for NeWCRFs Training
### NeWCRFs requires training files listing RGB-depth pairs. Generate train.txt:

In [17]:
# import os

# RGB_DATASET_PATH = "./datasets/custom_data/rgb_depth/"
# DEPTH_DATASET_PATH = "./datasets/custom_data/custom_data_converted/"

# TRAIN_FILE = "./datasets/custom_data/train.txt"

# # Collect image pairs
# with open(TRAIN_FILE, "w") as f:
#     for folder in os.listdir(DATASET_PATH):
#         rgb_folder = os.path.join(RGB_DATASET_PATH, folder, "*", "rgb")
#         depth_folder = os.path.join(DEPTH_DATASET_PATH, folder, "depth")

#         for file in os.listdir(rgb_folder):
#             rgb_path = os.path.join(rgb_folder, file)
#             depth_path = os.path.join(depth_folder, file)

#             if os.path.exists(depth_path):
#                 f.write(f"{rgb_path} {depth_path}\n")


In [18]:
# import os

# # Define paths
# RGB_DATASET_PATH = "./datasets/custom_data/rgb_depth/"
# DEPTH_DATASET_PATH = "./datasets/custom_data/custom_data_converted/"
# TRAIN_FILE = "./datasets/custom_data/train.txt"

# # Step 1: Collect all RGB files based on filename (ignore folder name)
# rgb_files = {}
# for folder in os.listdir(RGB_DATASET_PATH):  # Loop through RGB folders
#     rgb_folder = os.path.join(RGB_DATASET_PATH, folder, "rgb")
    
#     if os.path.exists(rgb_folder):
#         for file in os.listdir(rgb_folder):  # Collect filenames (without folder names)
#             rgb_files[file] = os.path.join(rgb_folder, file)  # Store full path

# # Step 2: Collect Depth files & match them using filename
# with open(TRAIN_FILE, "w") as f:
#     for folder in os.listdir(DEPTH_DATASET_PATH):  # Loop through Depth folders
#         depth_folder = os.path.join(DEPTH_DATASET_PATH, folder, "depth")
        
#         if os.path.exists(depth_folder):
#             for file in os.listdir(depth_folder):  # Collect depth filenames
#                 if file in rgb_files:  # Match by filename only
#                     f.write(f"{rgb_files[file]} {os.path.join(depth_folder, file)}\n")


In [22]:
import os

DATASET_PATH = "./datasets/custom_data/rgb_depth"
TRAIN_FILE = "./datasets/custom_data/train.txt"

# Create a list of RGB-Depth pairs
with open(TRAIN_FILE, "w") as f:
    for folder in os.listdir(DATASET_PATH):
        folder_path = os.path.join(DATASET_PATH, folder)
        
        rgb_folder = os.path.join(folder_path, "rgb")  # RGB folder
        depth_folder = os.path.join(folder_path, "depth_converted")  # Converted depth folder
        
        # Ensure both directories exist before processing
        if os.path.exists(rgb_folder) and os.path.exists(depth_folder):
            for file in os.listdir(rgb_folder):  # Iterate through RGB images
                rgb_path = os.path.join(rgb_folder, file)
                depth_path = os.path.join(depth_folder, file)

                if os.path.exists(depth_path):  # Ensure matching depth exists
                    f.write(f"{rgb_path} {depth_path}\n")

print("Train file successfully created with correct RGB-Converted Depth pairs!")


Train file successfully created with correct RGB-Converted Depth pairs!


## Automatically Split Your Data

In [ ]:
import os
import shutil
import random

DATASET_PATH = "./datasets/custom_data/rgb_depth/"
SPLIT_PATH = "./datasets/custom_data/custom_data_split/"
TRAIN_RATIO = 0.8  # 80% Train
VAL_RATIO = 0.1     # 10% Validation
TEST_RATIO = 0.1    # 10% Test

# Create output folders
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(SPLIT_PATH, split, "rgb"), exist_ok=True)
    os.makedirs(os.path.join(SPLIT_PATH, split, "depth_converted"), exist_ok=True)

# Collect all filenames
folders = os.listdir(DATASET_PATH)
rgb_depth_pairs = []
for folder in folders:
    rgb_folder = os.path.join(DATASET_PATH, folder, "rgb")
    depth_folder = os.path.join(DATASET_PATH, folder, "depth_converted")

    if os.path.exists(rgb_folder) and os.path.exists(depth_folder):
        for file in os.listdir(rgb_folder):
            depth_file = os.path.join(depth_folder, file)
            if os.path.exists(depth_file):
                rgb_depth_pairs.append((os.path.join(rgb_folder, file), depth_file))

# Shuffle and split data
random.shuffle(rgb_depth_pairs)
train_split = int(len(rgb_depth_pairs) * TRAIN_RATIO)
val_split = int(len(rgb_depth_pairs) * (TRAIN_RATIO + VAL_RATIO))

train_data = rgb_depth_pairs[:train_split]
val_data = rgb_depth_pairs[train_split:val_split]
test_data = rgb_depth_pairs[val_split:]

# Function to move files to correct folders
def move_files(data, split_name):
    for rgb_path, depth_path in data:
        shutil.copy(rgb_path, os.path.join(SPLIT_PATH, split_name, "rgb", os.path.basename(rgb_path)))
        shutil.copy(depth_path, os.path.join(SPLIT_PATH, split_name, "depth_converted", os.path.basename(depth_path)))

# Move files to respective folders
move_files(train_data, "train")
move_files(val_data, "val")
move_files(test_data, "test")

print("Data splitting completed! Check the 'datasets/custom_data_split' directory.")
